# Regression Week 5: Feature Selection and LASSO (Interpretation)

In this notebook, you will use LASSO to select features, building on a pre-implemented solver for LASSO (using GraphLab Create, though you can use other solvers). You will:
* Run LASSO with different L1 penalties.
* Choose best L1 penalty using a validation set.
* Choose best L1 penalty using a validation set, with additional constraint on the size of subset.

In the second notebook, you will implement your own LASSO solver, using coordinate descent. 

# Fire up graphlab create

In [1]:
import graphlab

# Load in house sales data

Dataset is from house sales in King County, the region where the city of Seattle, WA is located.

In [2]:
sales = graphlab.SFrame('kc_house_data.gl/')

[INFO] 1451556638 : INFO:     (initialize_globals_from_environment:282): Setting configuration variable GRAPHLAB_FILEIO_ALTERNATIVE_SSL_CERT_FILE to C:\Users\TDM\AppData\Local\Dato\Dato Launcher\lib\site-packages\certifi\cacert.pem
1451556638 : INFO:     (initialize_globals_from_environment:282): Setting configuration variable GRAPHLAB_FILEIO_ALTERNATIVE_SSL_CERT_DIR to 
This non-commercial license of GraphLab Create is assigned to pratyushdvd@gmail.com and will expire on September 29, 2016. For commercial licensing options, visit https://dato.com/buy/.

[INFO] Start server at: ipc:///tmp/graphlab_server-2544 - Server binary: C:\Users\TDM\AppData\Local\Dato\Dato Launcher\lib\site-packages\graphlab\unity_server.exe - Server log: C:\Users\TDM\AppData\Local\Temp\graphlab_server_1451556638.log.0
[INFO] GraphLab Server Version: 1.7.1


# Create new features

As in Week 2, we consider features that are some transformations of inputs.

In [3]:
from math import log, sqrt
sales['sqft_living_sqrt'] = sales['sqft_living'].apply(sqrt)
sales['sqft_lot_sqrt'] = sales['sqft_lot'].apply(sqrt)
sales['bedrooms_square'] = sales['bedrooms']*sales['bedrooms']

# In the dataset, 'floors' was defined with type string, 
# so we'll convert them to float, before creating a new feature.
sales['floors'] = sales['floors'].astype(float) 
sales['floors_square'] = sales['floors']*sales['floors']

* Squaring bedrooms will increase the separation between not many bedrooms (e.g. 1) and lots of bedrooms (e.g. 4) since 1^2 = 1 but 4^2 = 16. Consequently this variable will mostly affect houses with many bedrooms.
* On the other hand, taking square root of sqft_living will decrease the separation between big house and small house. The owner may not be exactly twice as happy for getting a house that is twice as big.

# Learn regression weights with L1 penalty

Let us fit a model with all the features available, plus the features we just created above.

In [4]:
all_features = ['bedrooms', 'bedrooms_square',
            'bathrooms',
            'sqft_living', 'sqft_living_sqrt',
            'sqft_lot', 'sqft_lot_sqrt',
            'floors', 'floors_square',
            'waterfront', 'view', 'condition', 'grade',
            'sqft_above',
            'sqft_basement',
            'yr_built', 'yr_renovated']
.19254432785, 661.7227177822587, 0.0, 15873.957259267981, 32.41022145125964, 690.1147733133256, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 2899.4202697498786, 30.011575302201045, 0.0, 0.0, 0.0]

Applying L1 penalty requires adding an extra parameter (`l1_penalty`) to the linear regression call in GraphLab Create. (Other tools may have separate implementations of LASSO.)  Note that it's important to set `l2_penalty=0` to ensure we don't introduce an additional L2 penalty.

In [5]:
model_all = graphlab.linear_regression.create(sales, target='price', features=all_features,
                                              validation_set=None, 
                                              l2_penalty=0., l1_penalty=1e10)

PROGRESS: Linear regression:
PROGRESS: --------------------------------------------------------
PROGRESS: Number of examples          : 21613
PROGRESS: Number of features          : 17
PROGRESS: Number of unpacked features : 17
PROGRESS: Number of coefficients    : 18
PROGRESS: Starting Accelerated Gradient (FISTA)
PROGRESS: --------------------------------------------------------
PROGRESS: +-----------+----------+-----------+--------------+--------------------+---------------+
PROGRESS: | Iteration | Passes   | Step size | Elapsed Time | Training-max_error | Training-rmse |
PROGRESS: +-----------+----------+-----------+--------------+--------------------+---------------+
PROGRESS: Tuning step size. First iteration could take longer than subsequent iterations.
PROGRESS: | 1         | 2        | 0.000002  | 1.484374     | 6962915.603493     | 426631.749026 |
PROGRESS: | 2         | 3        | 0.000002  | 1.578125     | 6843144.200219     | 392488.929838 |
PROGRESS: | 3         | 4      

Find what features had non-zero weight.

In [7]:
list(model_all.coefficients['value'])

[274873.0559504957,
 0.0,
 0.0,
 8468.531086910072,
 24.420720982445214,
 350.0605533860648,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 842.0680348976282,
 20.024722417091304,
 0.0,
 0.0,
 0.0]

Note that a majority of the weights have been set to zero. So by setting an L1 penalty that's large enough, we are performing a subset selection. 

***QUIZ QUESTION***:
According to this list of weights, which of the features have been chosen? 

# Selecting an L1 penalty

To find a good L1 penalty, we will explore multiple values using a validation set. Let us do three way split into train, validation, and test sets:
* Split our sales data into 2 sets: training and test
* Further split our training data into two sets: train, validation

Be *very* careful that you use seed = 1 to ensure you get the same answer!

In [8]:
(training_and_validation, testing) = sales.random_split(.9,seed=1) # initial train/test split
(training, validation) = training_and_validation.random_split(0.5, seed=1) # split training into train and validate

Next, we write a loop that does the following:
* For `l1_penalty` in [10^1, 10^1.5, 10^2, 10^2.5, ..., 10^7] (to get this in Python, type `np.logspace(1, 7, num=13)`.)
    * Fit a regression model with a given `l1_penalty` on TRAIN data. Specify `l1_penalty=l1_penalty` and `l2_penalty=0.` in the parameter list.
    * Compute the RSS on VALIDATION data (here you will want to use `.predict()`) for that `l1_penalty`
* Report which `l1_penalty` produced the lowest RSS on validation data.

When you call `linear_regression.create()` make sure you set `validation_set = None`.

Note: you can turn off the print out of `linear_regression.create()` with `verbose = False`

In [21]:
import numpy as np
for l1_penalty in np.logspace(1, 7, num=13):
    model_all = graphlab.linear_regression.create(training, target='price', features=all_features,
                                              validation_set=None, 
                                              l2_penalty=0., l1_penalty=l1_penalty,verbose= False)
    pred=model_all.predict(validation)
    rss=((((pred-validation['price'])**2)).sum())
    print rss
    

6.25766285142e+14
6.25766285362e+14
6.25766286058e+14
6.25766288257e+14
6.25766295212e+14
6.25766317206e+14
6.25766386761e+14
6.25766606749e+14
6.25767302792e+14
6.25769507644e+14
6.25776517727e+14
6.25799062845e+14
6.25883719085e+14


In [14]:
np.logspace(1, 7, num=13)

*** QUIZ QUESTIONS ***
1. What was the best value for the `l1_penalty`?
2. What is the RSS on TEST data of the model with the best `l1_penalty`?

In [46]:
print np.logspace(8, 10, num=20)

[  1.00000000e+08   1.27427499e+08   1.62377674e+08   2.06913808e+08
   2.63665090e+08   3.35981829e+08   4.28133240e+08   5.45559478e+08
   6.95192796e+08   8.85866790e+08   1.12883789e+09   1.43844989e+09
   1.83298071e+09   2.33572147e+09   2.97635144e+09   3.79269019e+09
   4.83293024e+09   6.15848211e+09   7.84759970e+09   1.00000000e+10]


***QUIZ QUESTION***
Also, using this value of L1 penalty, how many nonzero weights do you have?

In [30]:
model_all_best = graphlab.linear_regression.create(training, target='price', features=all_features,
                                              validation_set=None, 
                                              l2_penalty=0., l1_penalty=1e10,verbose= False)
print list(model_all_best.coefficients['value'])

[564482.1368441753, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]


# Limit the number of nonzero weights

What if we absolutely wanted to limit ourselves to, say, 7 features? This may be important if we want to derive "a rule of thumb" --- an interpretable model that has only a few features in them.

In this section, you are going to implement a simple, two phase procedure to achive this goal:
1. Explore a large range of `l1_penalty` values to find a narrow region of `l1_penalty` values where models are likely to have the desired number of non-zero weights.
2. Further explore the narrow region you found to find a good value for `l1_penalty` that achieves the desired sparsity.  Here, we will again use a validation set to choose the best value for `l1_penalty`.

In [32]:
max_nonzeros = 7

## Exploring the larger range of values to find a narrow range with the desired sparsity

Let's define a wide range of possible `l1_penalty_values`:

In [33]:
l1_penalty_values = np.logspace(8, 10, num=20)

Now, implement a loop that search through this space of possible `l1_penalty` values:

* For `l1_penalty` in `np.logspace(8, 10, num=20)`:
    * Fit a regression model with a given `l1_penalty` on TRAIN data. Specify `l1_penalty=l1_penalty` and `l2_penalty=0.` in the parameter list. When you call `linear_regression.create()` make sure you set `validation_set = None`
    * Extract the weights of the model and count the number of nonzeros. Save the number of nonzeros to a list.
        * *Hint: `model['coefficients']['value']` gives you an SArray with the parameters you learned.  If you call the method `.nnz()` on it, you will find the number of non-zero parameters!* 

In [35]:
for l1_penalty in np.logspace(8, 10, num=20):
    model_all = graphlab.linear_regression.create(training, target='price', features=all_features,
                                              validation_set=None, 
                                              l2_penalty=0., l1_penalty=l1_penalty,verbose= False)
    coeff=model_all['coefficients']['value']
    print coeff.nnz

<bound method SArray.nnz of dtype: float
Rows: 18
[25090.917367212027, 7789.177061100565, 847.5596869428746, 25234.209194458632, 39.03944596356858, 1117.3118955675666, -0.025686118239919543, 143.98899196976993, 20695.35923964388, 12466.690650291855, 568204.6445841154, 91066.94280879853, 6360.780926249808, 6139.2128056478905, 43.035829924605586, 118.94587495435255, 9.040401654020448, 48.61546730927457]>
<bound method SArray.nnz of dtype: float
Rows: 18
[26746.661936635508, 7743.979047849886, 822.358945250938, 25178.625930586946, 39.010718135253995, 1114.9107159232667, -0.018663073722782936, 142.51979784137154, 20545.867304697553, 12339.245250245602, 558930.2470718035, 90439.72185124886, 6288.009465538855, 6118.412320616764, 42.95755676813044, 117.99004260104351, 8.924085433195646, 46.56185624767168]>
<bound method SArray.nnz of dtype: float
Rows: 18
[28873.181016622384, 7691.047075686332, 790.9175796843559, 25115.27853451593, 38.98207881320865, 1112.2394146479912, -0.02473736058080104, 

Out of this large range, we want to find the two ends of our desired narrow range of `l1_penalty`.  At one end, we will have `l1_penalty` values that have too few non-zeros, and at the other end, we will have an `l1_penalty` that has too many non-zeros.  

More formally, find:
* The largest `l1_penalty` that has more non-zeros than `max_nonzero` (if we pick a penalty smaller than this value, we will definitely have too many non-zero weights)
    * Store this value in the variable `l1_penalty_min` (we will use it later)
* The smallest `l1_penalty` that has fewer non-zeros than `max_nonzero` (if we pick a penalty larger than this value, we will definitely have too few non-zero weights)
    * Store this value in the variable `l1_penalty_max` (we will use it later)


*Hint: there are many ways to do this, e.g.:*
* Programmatically within the loop above
* Creating a list with the number of non-zeros for each value of `l1_penalty` and inspecting it to find the appropriate boundaries.

In [47]:
l1_penalty_min =2.97635144e+09
l1_penalty_max =3.79269019e+09

***QUIZ QUESTIONS***

What values did you find for `l1_penalty_min` and`l1_penalty_max`? 

## Exploring the narrow range of values to find the solution with the right number of non-zeros that has lowest RSS on the validation set 

We will now explore the narrow region of `l1_penalty` values we found:

In [50]:
l1_penalty_values = np.linspace(l1_penalty_min,l1_penalty_max,20)
print l1_penalty_values

[  2.97635144e+09   3.01931664e+09   3.06228183e+09   3.10524703e+09
   3.14821223e+09   3.19117743e+09   3.23414262e+09   3.27710782e+09
   3.32007302e+09   3.36303822e+09   3.40600341e+09   3.44896861e+09
   3.49193381e+09   3.53489901e+09   3.57786420e+09   3.62082940e+09
   3.66379460e+09   3.70675980e+09   3.74972499e+09   3.79269019e+09]


* For `l1_penalty` in `np.linspace(l1_penalty_min,l1_penalty_max,20)`:
    * Fit a regression model with a given `l1_penalty` on TRAIN data. Specify `l1_penalty=l1_penalty` and `l2_penalty=0.` in the parameter list. When you call `linear_regression.create()` make sure you set `validation_set = None`
    * Measure the RSS of the learned model on the VALIDATION set

Find the model that the lowest RSS on the VALIDATION set and has sparsity *equal* to `max_nonzero`.

In [49]:

for l1_penalty in np.linspace(l1_penalty_min,l1_penalty_max,20):
    model_all = graphlab.linear_regression.create(training, target='price', features=all_features,
                                              validation_set=None, 
                                              l2_penalty=0., l1_penalty=l1_penalty,verbose= False)
    pred=model_all.predict(validation)
    rss=((((pred-validation['price'])**2)).sum())
    print rss
    coeff=model_all['coefficients']['value']
    print coeff.nnz

9.66925692362e+14
<bound method SArray.nnz of dtype: float
Rows: 18
[196100.9378056226, 2181.574321070697, 0.0, 17962.696661157814, 34.14246565122554, 789.319789078241, 0.0, 0.0, 3665.930817596536, 0.0, 0.0, 11333.841030833106, 0.0, 3578.900400438109, 32.74320137175829, 12.795381135944636, 0.0, 0.0]>
9.74019450085e+14
<bound method SArray.nnz of dtype: float
Rows: 18
[198563.24621808334, 2067.0151555582443, 0.0, 17810.387597798555, 34.021510305615884, 782.1823176948292, 0.0, 0.0, 3358.203305223814, 0.0, 0.0, 9876.737608119603, 0.0, 3528.2550088708495, 32.53723292119223, 11.05072885745476, 0.0, 0.0]>
9.81188367942e+14
<bound method SArray.nnz of dtype: float
Rows: 18
[201025.56040541263, 1952.4764896096917, 0.0, 17658.089496535536, 33.900565354624526, 775.0454285495368, 0.0, 0.0, 3050.4721940384243, 0.0, 0.0, 8418.98705902193, 0.0, 3477.612166173268, 32.331265598515046, 9.306267213386809, 0.0, 0.0]>
9.89328342459e+14
<bound method SArray.nnz of dtype: float
Rows: 18
[203443.60100002843,

***QUIZ QUESTIONS***
1. What value of `l1_penalty` in our narrow range has the lowest RSS on the VALIDATION set and has sparsity *equal* to `max_nonzeros`?
2. What features in this model have non-zero coefficients?

In [ ]:
1.04693748875e+15
<bound method SArray.nnz of dtype: float
Rows: 18
[222253.19254432785, 661.7227177822587, 0.0, 15873.957259267981, 32.41022145125964, 690.1147733133256, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 2899.4202697498786, 30.011575302201045, 0.0, 0.0, 0.0]